In [50]:
import pandas as pd
import re
from tqdm.notebook import tqdm

bungae_df = pd.read_csv('./new_bgzt_final_data.csv')
bungae_df['id'] = bungae_df['id'].astype(str)
bungae_df['cat_id'] = bungae_df['cat_id'].astype(str)
bungae_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203018 entries, 0 to 1203017
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1203018 non-null  object
 1   name            1203018 non-null  object
 2   image_url_list  1203018 non-null  object
 3   price           1203018 non-null  int64 
 4   info            1199641 non-null  object
 5   cat_id          1203018 non-null  object
 6   writed_at       1203018 non-null  object
 7   tag             1112801 non-null  object
 8   cat1            1203018 non-null  object
 9   cat2            1203018 non-null  object
 10  cat3            1133774 non-null  object
dtypes: int64(1), object(10)
memory usage: 101.0+ MB


In [51]:
# 새롭게 데이터 뽑기
from tqdm.notebook import tqdm
# -- 소분류 카테고리당 최대 700까지 줄이기 --
cat_id_list = list(bungae_df['cat_id'].unique())

new_df = pd.DataFrame()
for cat_id in tqdm(cat_id_list):
    tmp_df = bungae_df[bungae_df['cat_id'] == cat_id]
    if len(tmp_df) > 700: # 700개가 넘으면 700개만 가져오도록
        new_df = pd.concat([new_df, tmp_df.sample(700)], ignore_index=True)
    elif len(tmp_df) < 700: # 700개가 넘지 않으면 그대로 가져올 수 있도록
        new_df = pd.concat([new_df, tmp_df], ignore_index=True)

new_df = new_df.reset_index(drop=True)
print(len(new_df))

  0%|          | 0/157 [00:00<?, ?it/s]

109248


In [52]:
url_list = list()
for idx in tqdm(range(len(new_df))):
    tmp_url = new_df.loc[idx, 'image_url_list']
    url = re.sub('[\[\']', '', tmp_url.split(',')[0])
    
    url_list.append(url)

  0%|          | 0/109248 [00:00<?, ?it/s]

In [63]:
new_df['url'] = url_list

In [64]:
new_df = new_df[['id', 'name', 'cat_id', 'url']].copy()
new_df

,id,name,cat_id,url
0,207315989,루이비통패딩점퍼,310300200,https://media.bunjang.co.kr/product/207315989_...
1,213340343,플라스틱아일랜드 라쿤 후드 덕다운 롱패딩,310300200,https://media.bunjang.co.kr/product/213340343_...
2,179445605,**라쿤털 후드패딩,310300200,https://media.bunjang.co.kr/product/179445605_...
3,214139452,노이아고 NOI497 에센셜 다운 숏패딩,310300200,https://media.bunjang.co.kr/product/214139452_...
4,203871555,후아유 리버서블 숏패딩(플리스),310300200,https://media.bunjang.co.kr/product/203871555_...
...,...,...,...,...
109243,214388239,알라딘 셜록홈즈 금속 키링 열쇠고리 굿즈,400999,https://media.bunjang.co.kr/product/214388239_...
109244,81582461,새) 자주. 수납. 정리. 옷장 가방,400999,https://media.bunjang.co.kr/product/81582461_1...
109245,219590266,귀마개,400999,https://media.bunjang.co.kr/product/219590266_...
109246,225809818,하회탈 이매탈 팝니다,400999,https://media.bunjang.co.kr/product/225809818_...


In [66]:
# new_df -> to csv
new_df.to_csv('./v2_newly_reduced_bungae_df.csv', index=False)

In [68]:
# cat_id, url, id 가져와서 리스트로 만들기
cat_url_prd_list = list()
for idx in tqdm(range(len(new_df))):
    cat_id = new_df.loc[idx, 'cat_id']
    url = new_df.loc[idx, 'url']
    prd_id = new_df.loc[idx, 'id']
    cat_url_prd_list.append('__'.join([cat_id, url, prd_id]))

  0%|          | 0/109248 [00:00<?, ?it/s]

In [69]:
# 텍스트 파일로 바꿔주기
with open('./bungae_cat_url_prd_list.txt', 'w+') as file: 
    file.write(','.join(cat_url_prd_list))

In [70]:
cat_url_prd_list[:4]

['310300200__https://media.bunjang.co.kr/product/207315989_1_1681528252_w{res}.jpg__207315989',
 '310300200__https://media.bunjang.co.kr/product/213340343_1_1677694654_w{res}.jpg__213340343',
 '310300200__https://media.bunjang.co.kr/product/179445605_1_1657123055_w{res}.jpg__179445605',
 '310300200__https://media.bunjang.co.kr/product/214139452_1_1685294482_w{res}.jpg__214139452']

In [13]:
#  -- making folders --

In [71]:
# 1. folder 만들기 (maincat에 따라)

import os
import pandas as pd

def make_folders_for_image(df):
    cat_id_list = list(df['cat_id'].unique())
    
    for cat_id in cat_id_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'./bungae_images_v2/{cat_id}')
        
    return 

make_folders_for_image(new_df)

In [ ]:
# -- 2. 이미지 파일만 있는 데이터 로우 가져와서 새로운 데이터 프레임 만들어 주기--

In [72]:
import pandas as pd

df = pd.read_csv('./v2_newly_reduced_bungae_df.csv')
df['id'] = df['id'].astype(str)
df['cat_id'] = df['cat_id'].astype(str)

# 데이터 수정 전 len
print(len(df))

109248


In [74]:
# 열리지 않는 이미지를 가지고 있는 행 제거하기
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os

# 파일이 열리는 row의 인덱스만 가져오기
idx_list = list()
for idx in tqdm(range(len(df))):
    prd_id = df.loc[idx, 'id']
    cat_id = df.loc[idx, 'cat_id']

    image_path = f'./bungae_images_v2/{cat_id}/{prd_id}.jpg'
    if os.path.isfile(image_path):
        idx_list.append(idx)
    else:
        pass

  0%|          | 0/109248 [00:00<?, ?it/s]

In [77]:
# 수정 후 데이터 프레임 길이
# 약 600개 정도 사라짐

df = df.iloc[idx_list].copy()
print(len(df))
# 학습에 이용할 새로운 데이터
df.to_csv('./v3_newly_reduced_bungae_df.csv', index=False)

108647


In [ ]:
# -- 3. 이미지 파일 train, val에 따라서 폴더에 재배치 --

In [79]:
# 새로운 폴더 생성
import os
import pandas as pd

def make_train_val_folders(df):
    cat_id_list = list(df['cat_id'].unique())
    
    for cat_id in cat_id_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'./bungae_images_v3/train/{cat_id}')
        os.makedirs(f'./bungae_images_v3/val/{cat_id}')
        
    return 

make_train_val_folders(df)

In [80]:
# 데이터 분리하기
from sklearn.model_selection import train_test_split

def split_train_test_data(df):
    # -- train, test split --
    train_x, test_x, train_y, test_y = train_test_split(df['name'],
                                                        df['cat_id'].values,
                                                        random_state=42, 
                                                        test_size=.2)
    # -- train_df, test_df split --
    train_df, test_df = df.loc[train_x.index], df.loc[test_x.index]
    
    # -- resetting index -- 
    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    return train_df, test_df

train_df, test_df = split_train_test_data(df)


In [85]:
# 이미지 파일 train, val 따라 옮기기
# train
import shutil

for idx in tqdm(range(len(train_df))):
    prd_id = train_df.loc[idx, 'id']
    cat_id = train_df.loc[idx, 'cat_id']
    
    pre_image_path = f'./bungae_images_v2/{cat_id}/{prd_id}.jpg'
    after_image_path = f'./bungae_images_v3/train/{cat_id}/{prd_id}.jpg'
    
    # 파일 옮기기
    shutil.move(pre_image_path, after_image_path)

  0%|          | 0/86917 [00:00<?, ?it/s]

In [87]:
# val
for idx in tqdm(range(len(test_df))):
    prd_id = test_df.loc[idx, 'id']
    cat_id = test_df.loc[idx, 'cat_id']
    
    pre_image_path = f'./bungae_images_v2/{cat_id}/{prd_id}.jpg'
    after_image_path = f'./bungae_images_v3/val/{cat_id}/{prd_id}.jpg'
    
    # 파일 옮기기
    shutil.move(pre_image_path, after_image_path)

  0%|          | 0/21730 [00:00<?, ?it/s]